In [1]:
import plotly
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
import os
import volume as vol

folder_path = "/media/mfaria/Ganesha/20171219_backup_home_catec/Margarida/20190100_exploration/sim_data/valid"
#folder_path = "/home/mfaria/Flying_Octomap_code/src/data"
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(folder_path):
    for file in f:
        if "frontiers.csv" in file:
            files.append(os.path.join(r, file))
            

layout = go.Layout(
    title='Volume of known map',
    titlefont=dict(
        family='Courier New, monospace',
        size=24,
    ),
    xaxis=dict(
        title='Time in minutes',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        dtick=10
    ),
    yaxis=dict(
        title='Explored volume (m )',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

In [2]:
def maxTime(file_path):
    df = pd.read_csv(file_path)
    return df.time_ellapsed_millis.max()

millis = max([maxTime(f) for f in files]) ;

# Geofence volume
x_side = 70
y_side = 38
z_side = 31
max_volume_m = x_side*y_side*z_side
max_time = millis.max()
trace_max_volume = go.Scatter(
    x=[0, max_time/1000/60],
    y=[max_volume_m, max_volume_m],
    name='Geofence volume',
    mode='lines',
)

In [3]:
max_volume_m

82460

In [4]:
def maxExploredVolume(file_path):
    df = pd.read_csv(file_path)
    return (df.free +df.occupied).max()

maximum_explored_volume = max([maxExploredVolume(f) for f in files]) 
trace_max_explored_volume = go.Scatter(
    x=[0, max_time/1000/60],
    y=[maximum_explored_volume, maximum_explored_volume],
    name='Maximum explored volume',
    mode='lines',
)

In [5]:
def volumeDf(file_path, label):
    df = pd.read_csv(file_path)
    trace_free_volume = go.Scatter(
        x=df.time_ellapsed_millis/1000/60,
        y=df.free +df.occupied,
        name='Run ' + str(label),
        mode='lines'
    )
    return trace_free_volume

In [6]:
traces = []
label = 1
for f in files:
    traces.append(volumeDf(f, label))
    label = label +1
traces.append(trace_max_explored_volume)
traces.append(trace_max_volume)
fig=dict(data=traces, layout=layout)
plotly.offline.plot(fig, filename='./volume_exploration_vs_time.html', image='png')
plotly.offline.iplot(fig)
pio.write_image(fig, '/media/mfaria/Ganesha/20171219_backup_home_catec/Margarida/20190100_exploration/sim_data/valid/volume_all_samples.pdf', scale=2)

plotly.offline.plot(fig, filename='./volume_exploration_vs_time.html', image='png')

'file:///home/mfaria/Flying_Octomap_code/src/_generate_plots/Exploration/volume_exploration_vs_time.html'

In [7]:
def percentExplored(file_path, max):
    df = pd.read_csv(file_path)
    d = ((df.free +df.occupied).max() * 100) /  max
    print "Explored {} percent of the explorable volume.".format(d)
    return d
max_volume_explored_df = pd.DataFrame([percentExplored(f, max_volume_m) for f in files ])

Explored 98.1885520252 percent of the explorable volume.


In [8]:
def statistics(df):
    print "The average is {} +- {}".format(df.mean()[0], df.std()[0])
    print "The median is {} bounded by {} and {}".format(df.median()[0],  df.min()[0],  df.max()[0])
statistics(max_volume_explored_df)

The average is 98.1885520252 +- nan
The median is 98.1885520252 bounded by 98.1885520252 and 98.1885520252


In [9]:
for f in  files:
    df = pd.read_csv(f)
    total_reachable_volume = max(df.free +df.occupied)
    if (total_reachable_volume == maximum_explored_volume):
        last_row = df.tail(1)
        percent_free =  last_row.free.values[0] * 100 / maximum_explored_volume
        percent_occupied = last_row.occupied.values[0] * 100 / maximum_explored_volume

        print "Of the total explored space {} is free and  {} is occupied.".format(percent_free, percent_occupied)

Of the total explored space 97.7829288934 is free and  2.21697230007 is occupied.


In [10]:
for f in  files:
    df = pd.read_csv(f)
    total_reachable_volume = max(df.free +df.occupied)
    if (total_reachable_volume == maximum_explored_volume):
        last_row = df.tail(1)
        percent_free =  last_row.free.values[0] * 100 / max_volume_m
        percent_occupied = last_row.occupied.values[0] * 100 / max_volume_m
        reachable= (last_row.free.values[0] + last_row.occupied.values[0]) * 100 / max_volume_m
        known = (last_row.free.values[0] + last_row.occupied.values[0])
        print "Geofence: {}   Known: {}   Free: {}   Occupied: {}".format(max_volume_m, known, last_row.free.values[0], last_row.occupied.values[0])
        print "{} of the space is reachable. {} of the reachable space is free and  {} is occupied.".format(reachable, percent_free, percent_occupied)

Geofence: 82460   Known: 80966.2   Free: 79171.2   Occupied: 1795.0
98.1884550085 of the space is reachable. 96.0116420082 of the reachable space is free and  2.17681300024 is occupied.
